In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd

from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, ResNet50, decode_predictions
from tensorflow.keras.layers import Dense, LSTM,Embedding,Flatten,Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint

2024-06-24 21:33:39.218461: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
%load_ext tensorboard

In [ ]:
# !rm -rf ./logs/

In [2]:
import os
def png2jpg(path):
    for file in os.listdir(path):
        if file.endswith('.png'):
            base_name=os.path.splitext(file)[0]
            new_name= base_name+'.jpg'  
            os.rename(os.path.join(path,file), os.path.join(path,new_name))

png2jpg('/Users/Mehr/Desktop/Python/excercise/june/tree/tree')

In [ ]:
# # from tensorflow.keras.applications import InceptionV3
# # base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


# # from tensorflow.keras.applications import EfficientNetB0
# # base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# from tensorflow.keras.applications import Xception
# base_model = Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# #base_model=VGG16(weights='imagenet',include_top=False, input_shape=(224,224,3))

# x=base_model.output
# #x=Dropout(.1)(x)
# x=Flatten()(x)

# x=Dense(264,activation='relu')(x)
# predictions=Dense(1,activation='linear')(x)
# model=Model(inputs=base_model.input, outputs=predictions)


# # for layer in base_model.layers:
# #   layer.trainable=False

# #model_checkpoint_callback=ModelCheckpoint(filepath='trees.keras',monitor='val_loss',save_best_only=True)
# early_stopping_callback=EarlyStopping(monitor='val_loss', patience=3)

# tensorboard_callback =TensorBoard(log_dir='logs',histogram_freq=1)


# model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# history=model.fit(train_generator,
#                   validation_data=validation_generator,
#                   epochs=10,
#                   #steps_per_epoch=8,
#                   callbacks=[tensorboard_callback,model_checkpoint_callback])
#                   #early_stopping_callback])


In [7]:
!pip install scikeras   # gpu compute platform
#!pip install scikeras[tensorflow-cpu]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.4 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: keras
    Found existing installation: keras 3.1.1
    Uninstalling keras-3.1.1:
      Successfully uninstalled keras-3.1.1


In [8]:
#from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from scikeras.wrappers import KerasRegressor

In [ ]:
import numpy as np
#from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten
from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications import EfficientNetB0,EfficientNetB1,EfficientNetB2,EfficientNetB3

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import Xception



# Function to create model, required for KerasRegressor
def create_model(learning_rate, neurons):
    # Create an Adam optimizer with the given learning rate
    opt = Adam(learning_rate=learning_rate)

    #base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    x = base_model.output
    x = Flatten()(x)
    x = Dense(neurons, activation='relu')(x)
    predictions = Dense(1, activation='linear')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(loss='mean_squared_error', optimizer=opt)
    return model

def generator_to_data(generator):
    data_list = []
    labels_list = []
    for i in range(generator.samples // generator.batch_size):
        data, labels = next(generator)
        data_list.append(data)
        labels_list.append(labels)
    return np.concatenate(data_list), np.concatenate(labels_list)

# Assuming train_generator is defined elsewhere in your code
X_train, y_train = generator_to_data(train_generator)

# Create model
model = KerasRegressor(build_fn=create_model, verbose=0,
                       neurons=32,learning_rate=0.1,
                       )

# Define the grid search parameters
params = {
    'neurons': [264,512,1028],
    #'activation': ['relu', 'tanh'],
    'batch_size': [10,32, 128],
    'epochs': [10, 15,20],
    'learning_rate': [0.1, 0.01, 0.001]
}

# Search the grid
random_search = RandomizedSearchCV(estimator=model, param_distributions=params, n_iter=30, n_jobs=2, cv=3)

random_search_result = random_search.fit(X_train, y_train,verbose=True)

# Summarize results
print("Best: %f using %s" % (random_search_result.best_score_, random_search_result.best_params_))


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


KeyboardInterrupt: 

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:1052: UserWarning: One or more of the test scores are non-finite: [-1.56665672e+17 -1.61089643e+00            -inf]
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:1063: RuntimeWarning: invalid value encountered in subtract
  (array - array_means[:, np.newaxis]) ** 2, axis=1, weights=weights
/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
Best: -1.610896 using {'neurons': 264, 'learning_rate': 0.001, 'epochs': 15}

In [ ]:
%tensorboard --logdir=logs

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import os

# Load the CSV file containing image filenames and labels
labels = pd.read_csv("tree.csv")

# Set the number of augmented images you want to generate
num_augmented_images = 1000

# Define the image data generator with desired augmentations
datagen = ImageDataGenerator(
    horizontal_flip=True,
    width_shift_range=.1,
    zoom_range=.2,
    rotation_range=15,
    brightness_range=[.7, 1.3],
    shear_range=45,
    channel_shift_range=100,
    rescale=1.0/255
)

# Define the target size of the images
target_size = (224, 224)

# Define the directory where the original images are stored
data_dir = 'image/'

# Create a generator for the training data
train_generator = datagen.flow_from_dataframe(
    labels,
    directory=data_dir,
    x_col='filenames',
    y_col='tree_count',
    target_size=target_size,
    batch_size=1,  # Set to 1 to generate images one by one
    class_mode='raw',
    shuffle=True  # Shuffle the data
)

# Function to save generated images to disk
def save_augmented_images(generator, num_images, save_dir):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    filenames = []
    tree_counts = []
    for i in range(num_images):
        img, label = next(generator)
        img_path = os.path.join(save_dir, f"augmented_image_{i}.png")
        img = img[0]  # Remove batch dimension
        img = (img * 255).astype('uint8')  # Rescale back to 0-255
        Image.fromarray(img).save(img_path)
        filenames.append(f"augmented_image_{i}.png")
        tree_counts.append(label[0])
    return pd.DataFrame({'filenames': filenames, 'tree_count': tree_counts})

# Save augmented images to the specified directory
augmented_labels = save_augmented_images(train_generator, num_augmented_images, 'augmented_images')

# Save the augmented_labels DataFrame to a CSV file
augmented_labels.to_csv('augmented_labels.csv', index=False)


Found 23 validated image filenames.


In [ ]:
%tensorboard --logdir=logs

UsageError: Line magic function `%tensorboard` not found.


# A New Version

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 675.8/675.8 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.1/296.1 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 979.1/979.1 kB 50.4 MB/s eta 0:00:00
  Attempting uninstall: python-box
    Found existing installation: python-box 7.2.0
    Uninstalling python-box-7.2.0:
      Successfully uninstalled python-box-7.2.0


In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import os

# Load the CSV file containing image filenames and labels
labels = pd.read_csv("tree.csv")

# Set the number of augmented images you want to generate
num_augmented_images = 2000

# Define the image data generator with desired augmentations
datagen = ImageDataGenerator(
    horizontal_flip=True,
    width_shift_range=.1,
    zoom_range=.2,
    rotation_range=15,
    brightness_range=[.7, 1.3],
    shear_range=45,
    channel_shift_range=100,
    rescale=1.0/255
)

# Define the target size of the images
target_size = (224, 224)

# Define the directory where the original images are stored
data_dir = 'image/'

# Create a generator for the training data
train_generator = datagen.flow_from_dataframe(
    labels,
    directory=data_dir,
    x_col='filenames',
    y_col='tree_count',
    target_size=target_size,
    batch_size=1,  # Set to 1 to generate images one by one
    class_mode='raw',
    shuffle=True  # Shuffle the data
)

# Function to save generated images to disk
def save_augmented_images(generator, num_images, save_dir):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    filenames = []
    tree_counts = []
    for i in range(num_images):
        img, label = next(generator)
        img_path = os.path.join(save_dir, f"augmented_image_{i}.png")
        img = img[0]  # Remove batch dimension
        img = (img * 255).astype('uint8')  # Rescale back to 0-255
        Image.fromarray(img).save(img_path)
        filenames.append(f"augmented_image_{i}.png")
        tree_counts.append(label[0])
    return pd.DataFrame({'filenames': filenames, 'tree_count': tree_counts})

# Save augmented images to the specified directory
augmented_labels = save_augmented_images(train_generator, num_augmented_images, 'augmented_images')

# Save the augmented_labels DataFrame to a CSV file
augmented_labels.to_csv('augmented_labels.csv', index=False)


Found 41 validated image filenames.


In [ ]:
from tensorflow.keras.layers import Input, DepthwiseConv2D, BatchNormalization

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout, Rescaling, ZeroPadding2D
from tensorflow.keras.regularizers import l2


def create_model2():
    input_layer = Input(shape=(224, 224, 3))
    x = Rescaling(1./255)(input_layer)
    x = ZeroPadding2D(padding=(2, 2))(x)
    x = Conv2D(32, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)
    x = Conv2D(256, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)
    x = Flatten()(x)
    x = Dropout(0.5)(x)
    x = Dense(264, activation='relu', kernel_regularizer=l2(0.001))(x)
    x = Dropout(0.5)(x)
    predictions = Dense(1, activation='linear')(x)

    return Model(inputs=input_layer, outputs=predictions)




import tensorflow as tf
from tensorflow.keras import layers

def create_model(input_shape=(224, 224, 3)):
    """
    Creates a lightweight convolutional neural network (CNN) for tree counting.

    Args:
        input_shape (tuple, optional): Input image shape (height, width, channels).
            Defaults to (224, 224, 3).

    Returns:
        tf.keras.Model: Compiled CNN model for tree counting.
    """

    inputs = layers.Input(shape=input_shape)

    # Block 1: Feature extraction with efficient depth-wise convolutions
    x = layers.Conv2D(filters=32, kernel_size=3, strides=1, padding='same', use_bias=False)(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.DepthwiseConv2D(kernel_size=3, strides=1, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPooling2D(pool_size=2, strides=2)(x)  # Reduce spatial resolution

    # Block 2: Feature extraction with depth-wise and point-wise convolutions
    x = layers.Conv2D(filters=64, kernel_size=3, strides=1, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.DepthwiseConv2D(kernel_size=3, strides=1, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(filters=64, kernel_size=1, strides=1, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPooling2D(pool_size=2, strides=2)(x)  # Reduce spatial resolution

    # Block 3: Feature extraction with mixed convolutions
    x = layers.Conv2D(filters=128, kernel_size=3, strides=1, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(filters=128, kernel_size=1, strides=1, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.DepthwiseConv2D(kernel_size=3, strides=1, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPooling2D(pool_size=2, strides=2)(x)  # Reduce spatial resolution

    # Block 4: Feature extraction with efficient convolutions
    x = layers.Conv2D(filters=256, kernel_size=3, strides=1, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(filters=256, kernel_size=1, strides=1, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.DepthwiseConv2D(kernel_size=3, strides=1, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    # Flatten for dense layers
    x = layers.GlobalAveragePooling2D()(x)

    # Dense layers for regression (adjust final activation based on task)
    outputs = layers.Dense(1, activation='linear')(x)  # Linear activation for regression

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model
    # Compile with appropriate


In [ ]:
%time
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0,MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten,Dropout,DepthwiseConv2D,Input,GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from sklearn.model_selection import train_test_split
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.utils import plot_model
from tensorflow.keras.applications import InceptionV3
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))



# Load the augmented labels CSV file
augmented_labels = pd.read_csv('augmented_labels.csv')

# Split the data into training and validation sets
train_labels, val_labels = train_test_split(augmented_labels, test_size=0.22, random_state=42)

# Define the image data generator for rescaling (no further augmentation needed for training and validation)
datagen = ImageDataGenerator(rescale=1.0/255)

# Create a generator for the augmented training data
train_generator = datagen.flow_from_dataframe(
    train_labels,
    directory='augmented_images',
    x_col='filenames',
    y_col='tree_count',
    target_size=(224, 224),
    batch_size=128,  # Use a larger batch size for training
    class_mode='raw',
    shuffle=True
)

# Create a generator for the validation data
validation_generator = datagen.flow_from_dataframe(
    val_labels,
    directory='augmented_images',
    x_col='filenames',
    y_col='tree_count',
    target_size=(224, 224),
    batch_size=32,
    class_mode='raw',
    shuffle=True
)

#base_model=create_model()


# Model
#InceptionV3

inputs = Input(shape=(224, 224, 3))
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

x = base_model(inputs)

x = base_model.output
x = GlobalAveragePooling2D()(x)
#x = layers.Dense(256, activation='relu')(x)
x = Flatten()(x)
x = Dense(256, activation='relu', kernel_regularizer=l2(0.001))(x)

predictions = Dense(1, activation='linear')(x)
model = Model(inputs=base_model.input, outputs=predictions)

#print(model.summary())
#plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)


model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

#CallBacks
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=3)
tensorboard_callback = TensorBoard(log_dir='logs', histogram_freq=1)
reduce_lr_callback = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.0000001)

# Train the model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=15,
    steps_per_epoch=train_generator.n // train_generator.batch_size,
    validation_steps=validation_generator.n // validation_generator.batch_size,
    callbacks=[tensorboard_callback,reduce_lr_callback]#, early_stopping_callback]
)



CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 5.25 µs
87910968/87910968 [==============================] - 0s 0us/step
Found 1560 validated image filenames.
Found 440 validated image filenames.
9406464/9406464 [==============================] - 0s 0us/step
Epoch 1/15
12/12 [==============================] - 100s 8s/step - loss: 772.0406 - val_loss: 486.7694 - lr: 0.0010
Epoch 2/15
12/12 [==============================] - 111s 10s/step - loss: 492.2406 - val_loss: 338.5053 - lr: 0.0010
Epoch 3/15
12/12 [==============================] - 96s 8s/step - loss: 354.5630 - val_loss: 277.1121 - lr: 0.0010
Epoch 4/15
12/12 [==============================] - 96s 8s/step - loss: 267.4743 - val_loss: 215.0968 - lr: 0.0010
Epoch 5/15
12/12 [==============================] - 94s 8s/step - loss: 201.7211 - val_loss: 190.6493 - lr: 0.0010
Epoch 6/15
12/12 [==============================] - 98s 8s/step - loss: 170.0570 - val_loss: 156.7651 - lr: 0.0010
Epoch 7/15
12/12 [======================

In [ ]:
#model.summary()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
model.save('inception_epoch_30.h5')

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import  load_img,img_to_array
from PIL import Image

with Image.open('test_image/test.jpg') as img:
  test_image=img.resize((224,224))
  test_image = test_image.convert('RGB')
#test_image=load_img('image/test.jpg')
plt.imshow(test_image)

test_image_array=img_to_array(test_image)
test_image_array = test_image_array / 255.0
print(test_image_array.shape)
test_image_array = np.expand_dims(test_image_array, axis=0)
print(test_image_array.shape)


test_image_array = preprocess_input(test_image_array)
predictions = model.predict(test_image_array)
print("the prediction of tree count : ",predictions)
print(' '*60)
print(f'Prediction: {round(predictions[0][0])}')


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.efficientnet import preprocess_input
from PIL import Image

# Load your trained model
#model2 = load_model('my_model1.h5')

# Load and preprocess the test image
def load_and_preprocess_image(img_path, target_size=(224, 224)):
    with Image.open(img_path) as img:
        img = img.resize(target_size)
        img = img.convert('RGB')  # Ensure it's in RGB mode
    img_array = img_to_array(img)
    img_array = img_array / 255.0  # Rescale pixel values
    img_array = np.expand_dims(img_array, axis=0)
    return preprocess_input(img_array)  # Use preprocess_input from the corresponding application module

# Path to your test image
img_path = 'test_image/test.jpg'

# Preprocess the image
test_image_array = load_and_preprocess_image(img_path)

# Display the image
plt.imshow(load_img(img_path))
plt.axis('off')
plt.show()

# Print the shape of the image array
print(test_image_array.shape)

# Make predictions
predictions = model.predict(test_image_array)

# Print the prediction
print(f'The prediction of tree count: {round(predictions[0][0])}')


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Evaluate the model on the validation set
val_loss = model.evaluate(validation_generator, steps=validation_generator.n // validation_generator.batch_size)
print(f'Validation Loss: {val_loss}')

# Make predictions on the validation set
val_predictions = model.predict(validation_generator, steps=validation_generator.n // validation_generator.batch_size)
val_true = val_labels['tree_count'][:len(val_predictions)]

# Compute and print metrics
mse = mean_squared_error(val_true, val_predictions)
mae = mean_absolute_error(val_true, val_predictions)
print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')

In [ ]:
%time
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0,MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten,Dropout,DepthwiseConv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from sklearn.model_selection import train_test_split
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.utils import plot_model
from tensorflow.keras.applications import InceptionV3
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))



# Load the augmented labels CSV file
augmented_labels = pd.read_csv('augmented_labels.csv')

# Split the data into training and validation sets
train_labels, val_labels = train_test_split(augmented_labels, test_size=0.22, random_state=42)

# Define the image data generator for rescaling (no further augmentation needed for training and validation)
datagen = ImageDataGenerator(rescale=1.0/255)

# Create a generator for the augmented training data
train_generator = datagen.flow_from_dataframe(
    train_labels,
    directory='augmented_images',
    x_col='filenames',
    y_col='tree_count',
    target_size=(224, 224),
    batch_size=128,  # Use a larger batch size for training
    class_mode='raw',
    shuffle=True
)

# Create a generator for the validation data
validation_generator = datagen.flow_from_dataframe(
    val_labels,
    directory='augmented_images',
    x_col='filenames',
    y_col='tree_count',
    target_size=(224, 224),
    batch_size=32,
    class_mode='raw',
    shuffle=True
)

#base_model=create_model()


# Model
#InceptionV3

inputs = layers.Input(shape=(224, 224, 3))
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

x = base_model(inputs)

x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation='relu')(x)
x = Flatten()(x)
x = Dense(256, activation='relu', kernel_regularizer=l2(0.001))(x)

predictions = Dense(1, activation='linear')(x)
model = Model(inputs=base_model.input, outputs=predictions)

#print(model.summary())
#plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)


model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

#CallBacks
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=3)
tensorboard_callback = TensorBoard(log_dir='logs', histogram_freq=1)
reduce_lr_callback = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.0000001)

# Train the model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=30,
    steps_per_epoch=train_generator.n // train_generator.batch_size,
    validation_steps=validation_generator.n // validation_generator.batch_size,
    callbacks=[tensorboard_callback,reduce_lr_callback]#, early_stopping_callback]
)



CPU times: user 4 µs, sys: 2 µs, total: 6 µs
Wall time: 34.6 µs
Found 1950 validated image filenames.
Found 550 validated image filenames.
16705208/16705208 [==============================] - 0s 0us/step
Epoch 1/50
15/15 [==============================] - 272s 18s/step - loss: 800.4761 - val_loss: 655.5438 - lr: 0.0010
Epoch 2/50
15/15 [==============================] - 223s 16s/step - loss: 633.7009 - val_loss: 598.1385 - lr: 0.0010
Epoch 3/50
15/15 [==============================] - 260s 18s/step - loss: 607.0005 - val_loss: 576.8295 - lr: 0.0010
Epoch 4/50
15/15 [==============================] - 262s 18s/step - loss: 610.0582 - val_loss: 563.5933 - lr: 0.0010
Epoch 5/50
15/15 [==============================] - 261s 18s/step - loss: 600.1154 - val_loss: 562.5284 - lr: 0.0010
Epoch 6/50
15/15 [==============================] - 228s 15s/step - loss: 592.2721 - val_loss: 570.3102 - lr: 0.0010
Epoch 7/50
15/15 [==============================] - 227s 15s/step - loss: 594.1062 - val_loss:

In [ ]:
%time
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0,MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten,Dropout,DepthwiseConv2D, Input,GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from sklearn.model_selection import train_test_split
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.utils import plot_model
from tensorflow.keras.applications import InceptionV3
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))



# Load the augmented labels CSV file
augmented_labels = pd.read_csv('augmented_labels.csv')

# Split the data into training and validation sets
train_labels, val_labels = train_test_split(augmented_labels, test_size=0.22, random_state=42)

# Define the image data generator for rescaling (no further augmentation needed for training and validation)
datagen = ImageDataGenerator(rescale=1.0/255)

# Create a generator for the augmented training data
train_generator = datagen.flow_from_dataframe(
    train_labels,
    directory='augmented_images',
    x_col='filenames',
    y_col='tree_count',
    target_size=(224, 224),
    batch_size=128,  # Use a larger batch size for training
    class_mode='raw',
    shuffle=True
)

# Create a generator for the validation data
validation_generator = datagen.flow_from_dataframe(
    val_labels,
    directory='augmented_images',
    x_col='filenames',
    y_col='tree_count',
    target_size=(224, 224),
    batch_size=32,
    class_mode='raw',
    shuffle=True
)

#base_model=create_model()


# Model
#InceptionV3

inputs = Input(shape=(224, 224, 3))
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

x = base_model(inputs)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Flatten()(x)
x = Dense(256, activation='relu', kernel_regularizer=l2(0.001))(x)

predictions = Dense(1, activation='linear')(x)
model = Model(inputs=base_model.input, outputs=predictions)

#print(model.summary())
#plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)


model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

#CallBacks
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=3)
tensorboard_callback = TensorBoard(log_dir='logs', histogram_freq=1)
reduce_lr_callback = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.0000001)

# Train the model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=30,
    steps_per_epoch=train_generator.n // train_generator.batch_size,
    validation_steps=validation_generator.n // validation_generator.batch_size,
    callbacks=[tensorboard_callback,reduce_lr_callback]#, early_stopping_callback]
)



CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 9.06 µs
87910968/87910968 [==============================] - 1s 0us/step
Found 1950 validated image filenames.
Found 550 validated image filenames.
Epoch 1/30
15/15 [==============================] - 313s 21s/step - loss: 702.6786 - val_loss: 508.4851 - lr: 0.0010
Epoch 2/30
15/15 [==============================] - 320s 23s/step - loss: 437.9388 - val_loss: 335.7757 - lr: 0.0010
Epoch 3/30
15/15 [==============================] - 322s 22s/step - loss: 265.5882 - val_loss: 230.3602 - lr: 0.0010
Epoch 4/30
15/15 [==============================] - 311s 21s/step - loss: 168.7386 - val_loss: 165.6201 - lr: 0.0010
Epoch 5/30
15/15 [==============================] - 321s 22s/step - loss: 129.7594 - val_loss: 134.6769 - lr: 0.0010
Epoch 6/30
15/15 [==============================] - 325s 22s/step - loss: 102.3752 - val_loss: 109.0508 - lr: 0.0010
Epoch 7/30
15/15 [==============================] - 320s 22s/step - loss: 81.5673 - val_loss: 

Num GPUs Available:  0
